In [11]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import math

In [12]:
# Load the test data
test_df_orig = pd.read_parquet(f"../data/test_data.parquet")
test_pred_cols = list(
    filter(
        lambda x: x.startswith("call") or x.startswith("put"),
        test_df_orig.columns.tolist(),
    )
)
# output_matching = pd.read_csv(f"matching/output_smooth.csv", header=None)
# output_matching.columns = test_pred_cols
test_df = test_df_orig.copy()
# test_df[test_pred_cols] = output_matching
print(test_df.shape)
print(list(test_df.columns))

test_df.head()

(12065, 96)
['timestamp', 'underlying', 'call_iv_24000', 'call_iv_24100', 'call_iv_24200', 'call_iv_24300', 'call_iv_24400', 'call_iv_24500', 'call_iv_24600', 'call_iv_24700', 'call_iv_24800', 'call_iv_24900', 'call_iv_25000', 'call_iv_25100', 'call_iv_25200', 'call_iv_25300', 'call_iv_25400', 'call_iv_25500', 'call_iv_25600', 'call_iv_25700', 'call_iv_25800', 'call_iv_25900', 'call_iv_26000', 'call_iv_26100', 'call_iv_26200', 'call_iv_26300', 'call_iv_26400', 'call_iv_26500', 'put_iv_23000', 'put_iv_23100', 'put_iv_23200', 'put_iv_23300', 'put_iv_23400', 'put_iv_23500', 'put_iv_23600', 'put_iv_23700', 'put_iv_23800', 'put_iv_23900', 'put_iv_24000', 'put_iv_24100', 'put_iv_24200', 'put_iv_24300', 'put_iv_24400', 'put_iv_24500', 'put_iv_24600', 'put_iv_24700', 'put_iv_24800', 'put_iv_24900', 'put_iv_25000', 'put_iv_25100', 'put_iv_25200', 'put_iv_25300', 'put_iv_25400', 'put_iv_25500', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 

,timestamp,underlying,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,0,24735.9,0.280939,NaN,NaN,NaN,0.242149,NaN,0.232439,NaN,...,0.006587,0.002826,-1.737617e+05,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06
1,1,24766.7,0.270276,NaN,0.258893,NaN,NaN,NaN,0.233548,NaN,...,0.005777,0.004588,-3.195086e+05,-0.024106,-0.004696,-0.000158,-4.529075e+06,-1.619022e+06,-0.956928,4.624907e+06
2,2,24896.9,NaN,0.251731,NaN,NaN,0.214869,0.20458,0.194604,NaN,...,0.000829,-0.034944,1.027525e+06,0.028201,0.032234,0.007687,2.663908e+06,0.000000e+00,-0.116264,-2.669766e+06
3,3,24898.1,0.241888,NaN,0.220505,NaN,0.198602,0.18619,NaN,NaN,...,-0.009323,-0.022969,-4.720074e+06,-0.001513,0.008704,-0.000206,-7.672936e+06,-1.903406e+06,-2.249208,-1.858254e+07
4,4,24906.5,0.235328,NaN,0.222983,0.214126,NaN,NaN,0.192603,NaN,...,-0.018674,-0.007588,-4.051681e+05,-0.136267,0.002425,-0.002200,1.334469e+06,2.548789e+05,1.999104,3.024212e+06


In [13]:
test_df.isna().sum().sum()

376504

In [14]:
pred_cols = [col for col in test_df.columns if col.startswith("call") or col.startswith("put")]
call_cols = [col for col in pred_cols if col.startswith("call")]
put_cols = [col for col in pred_cols if col.startswith("put")]

In [15]:
# For each row, fill missing call/put values with their counterpart for the same strike price
strikes = list(set([col.split('_')[-1] for col in pred_cols if '_' in col]))
for index, row in test_df.iterrows():
  for strike in strikes:
    call_col = f'call_iv_{strike}'
    put_col = f'put_iv_{strike}'
    
    if call_col in test_df.columns and put_col in test_df.columns:
      # If call is NaN but put is not, fill call with put value
      if pd.isna(row[call_col]) and not pd.isna(row[put_col]):
        test_df.at[index, call_col] = row[put_col]
      # If put is NaN but call is not, fill put with call value
      elif pd.isna(row[put_col]) and not pd.isna(row[call_col]):
        test_df.at[index, put_col] = row[call_col]

test_df.isna().sum().sum()

283769

In [16]:
test_df.to_csv("../data/test_putcall.csv", index=False)

In [17]:
test_pred = test_df[pred_cols].copy()
test_pred.to_csv("matching/input_putcall.csv", index=False, header=False)